In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

In [2]:
driver = webdriver.Ie('D:/Programming/IEDriverServer2.exe')
time.sleep(1)
driver.get('https://www.scourt.go.kr/portal/information/finalruling/peruse/peruse_status.jsp')
time.sleep(1)

iframes = driver.find_elements_by_tag_name('iframe')
print('현재 페이지에 iframe은 %d개가 있습니다.' % len(iframes))
driver.switch_to.frame(iframes[0])

driver.find_element_by_xpath('''//input[@id='accept']''').click()
time.sleep(1)
driver.find_element_by_xpath('''//input[@id='name']''').send_keys('박동주')
time.sleep(1)
driver.find_element_by_xpath('''//input[@id='jumin1']''').send_keys('951030')
time.sleep(1)
driver.find_element_by_xpath('''//input[@id='jumin2']''').send_keys('1126827')
time.sleep(1)
driver.find_element_by_xpath('''//img[@id='auth']''').click()
time.sleep(1)
driver.find_element_by_xpath('''//input[@id='q']''').send_keys('폭행 | 폭력')
time.sleep(1)
driver.find_element_by_xpath('''//input[@id='search-button']''').click()
time.sleep(1)
driver.find_element_by_xpath('''//select[@id='sviewlist']''').click()
time.sleep(1)
driver.find_element_by_xpath('''//select[@id='sviewlist']/option[@value='50']''').click()
time.sleep(1)
driver.find_element_by_xpath('''//dd[@class='btn']/img[@onclick='sview()']''').click()
time.sleep(1)


soup = BeautifulSoup(driver.page_source, 'html.parser')
alist = []
blist = []

for n,i in enumerate(soup.find_all('td')):
    j = i.get_text()
    j = " ".join(j.split())
    alist.append(j)
    if ((n+1) % 6) == 0:
        blist.append(alist)
        alist = []

num = 2

while num < 83:
    
    path = '''//a[@onclick="javascript:detail_search('{}');"]'''.format(num)
    
    if driver.find_element_by_xpath(path):
        driver.find_element_by_xpath(path).click()
        time.sleep(1)
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        if soup.find('tr',{'class':'disable2'}):
            soup.find('tr',{'class':'disable2'}).decompose()

        for n,i in enumerate(soup.find_all('td')):
            j = i.get_text()
            j = " ".join(j.split())
            alist.append(j)
            if ((n+1) % 6) == 0:
                blist.append(alist)
                alist = []
        num = num + 1
        
    else:
        continue
    
df = pd.DataFrame(blist)
time.sleep(3)
print('완료')
driver.quit()

현재 페이지에 iframe은 1개가 있습니다.
완료


In [3]:
df.rename(columns={0:'공백', 1:'선고일자',2:'법원명과 사건번호', 3:'사건명', 4:'주문', 5:'이유'}, inplace=True)
df.drop(columns='공백', inplace=True)

In [4]:
df

,선고일자,법원명과 사건번호,사건명,주문,이유
0,2020.10.23,광주지방법원 목포지원 [2020고정107],특수협박등[형사],.... 위 벌금액 상당의 가납을 명한다. 이 사건 공소사실 중 폭행의 점에 대한 ...,"... 경찰진술조서 1. 혼인관계증명서, 피해자 폭행 부위 사진ㆍ현장사진ㆍ위험한 물..."
1,2020.10.23,광주지방법원 목포지원 [2020고단547],특수상해[형사],"피고인을 징역 6월에 처한다. 다만, 이 판결 확정일로부터 2년간 위 형의 집행을 ...",... 따른 권고형의 범위 [유형의 결정] 폭력범죄 > 02. 특수상해ㆍ누범상해 >...
2,2020.10.22,제주지방법원 [2020고정468],재물손괴[형사],피고인을 벌금 300만 원에 처한다. 피고인이 위 벌금을 납입하지 아니하는 경우 금...,... 11. 13:00경 제주시 B에 있는 C지구대에서 가정폭력 관련 상담을 하던...
3,2020.10.22,광주지방법원 [2020노1868],재물손괴등[형사],피고인의 항소를 기각한다.,"... 동종 전과가 있고, 2017년 이후 여러 차례 폭행 범행을 저질렀으나 합의되..."
4,2020.10.22,광주지방법원 [2020고단3990],특수상해[형사],"피고인을 징역 8월에 처한다. 다만, 이 판결 확정일로부터 2년간 위 형의 집행을 ...",... 2년 피고인에게 폭력 범행으로... 피해자에게 가한 폭행의 방법과 정도가.....
...,...,...,...,...,...
4080,2020.08.19,수원지방법원 [2019노6420],강제추행등[형사],검사의 항소를 기각한다.,"... 피해자를 원심 판시 범죄사실과 같이 폭행하였고, 피해자, 피해자의 모친, 피..."
4081,,2020.08.19,서울중앙지방법원 [2019노3728],특정범죄가중···[형사],- - -
4082,해당 판결서는 열람이 제한되었습니다.,,2020.08.19,서울중앙지방법원 [2019노3571],업무방해등[형사]
4083,"..., 유형적이든 무형적이든 묻지 아니하며, 폭행ㆍ협박은 물론 사회적, 경제적, ...",,2020.08.19,대전지방법원 [2019노2622],공무집행방해[형사]


In [6]:
df2 = df.drop([4081,4082,4083,4084])
df2

,선고일자,법원명과 사건번호,사건명,주문,이유
0,2020.10.23,광주지방법원 목포지원 [2020고정107],특수협박등[형사],.... 위 벌금액 상당의 가납을 명한다. 이 사건 공소사실 중 폭행의 점에 대한 ...,"... 경찰진술조서 1. 혼인관계증명서, 피해자 폭행 부위 사진ㆍ현장사진ㆍ위험한 물..."
1,2020.10.23,광주지방법원 목포지원 [2020고단547],특수상해[형사],"피고인을 징역 6월에 처한다. 다만, 이 판결 확정일로부터 2년간 위 형의 집행을 ...",... 따른 권고형의 범위 [유형의 결정] 폭력범죄 > 02. 특수상해ㆍ누범상해 >...
2,2020.10.22,제주지방법원 [2020고정468],재물손괴[형사],피고인을 벌금 300만 원에 처한다. 피고인이 위 벌금을 납입하지 아니하는 경우 금...,... 11. 13:00경 제주시 B에 있는 C지구대에서 가정폭력 관련 상담을 하던...
3,2020.10.22,광주지방법원 [2020노1868],재물손괴등[형사],피고인의 항소를 기각한다.,"... 동종 전과가 있고, 2017년 이후 여러 차례 폭행 범행을 저질렀으나 합의되..."
4,2020.10.22,광주지방법원 [2020고단3990],특수상해[형사],"피고인을 징역 8월에 처한다. 다만, 이 판결 확정일로부터 2년간 위 형의 집행을 ...",... 2년 피고인에게 폭력 범행으로... 피해자에게 가한 폭행의 방법과 정도가.....
...,...,...,...,...,...
4076,2020.08.19,대전지방법원 천안지원 [2020고합53],강간미수[형사],... 집행을 유예한다. 피고인에게 40시간의 성폭력 치료강의 수강 및 200시간의...,".... 따라서 피고인이 폭행으로 피해자를... 경우, 피고인은 성폭력범죄의 처벌 ..."
4077,2020.08.19,서울중앙지방법원 [2019고정1153],재물손괴등[형사],피고인을 벌금 130만 원에 처한다. 위 벌금을 납입하지 아니하는 경우 10만 원을...,"... 법정진술 1. F 작성의 진술서 1. 발생보고(폭행, 특수폭행), 112사건..."
4078,2020.08.19,광주지방법원 순천지원 [2019고정459],폭행[형사],"피고인을 벌금 700,000원에 처한다. 피고인이 위 벌금을 납입하지 아니하는 경우...",... 피해자의 왼쪽 뺨을 1회 때리는 등 폭행하였다. 증거의 요지 1. 피고인의 ...
4079,2020.08.19,춘천지방법원 속초지원 [2019고정79],재물손괴[형사],"피고인을 벌금 2,000,000원에 처한다. 피고인이 위 벌금을 납입하지 아니하는 ...","..., 동생인 피해자 C가 피고인이 모친에 대한 폭력 행사 여부를 확인하기 위하여..."


In [8]:
df2.to_csv('law_crawling30.csv', sep=',', na_rep='NaN', encoding='utf-8')

In [9]:
tq = pd.read_csv("D:/Programming/2020.11.AIcompet/law_crawling31.csv", sep=",", encoding="CP949")
tq

,Unnamed: 0,선고일자,법원명과 사건번호,사건명,주문,이유
0,0,2020.10.23,광주지방법원 목포지원 [2020고정107],특수협박등[형사],.... 위 벌금액 상당의 가납을 명한다. 이 사건 공소사실 중 폭행의 점에 대한 ...,"... 경찰진술조서 1. 혼인관계증명서, 피해자 폭행 부위 사진ㆍ현장사진ㆍ위험한 물..."
1,1,2020.10.23,광주지방법원 목포지원 [2020고단547],특수상해[형사],"피고인을 징역 6월에 처한다. 다만, 이 판결 확정일로부터 2년간 위 형의 집행을 ...",... 따른 권고형의 범위 [유형의 결정] 폭력범죄 > 02. 특수상해ㆍ누범상해 >...
2,2,2020.10.22,제주지방법원 [2020고정468],재물손괴[형사],피고인을 벌금 300만 원에 처한다. 피고인이 위 벌금을 납입하지 아니하는 경우 금...,... 11. 13:00경 제주시 B에 있는 C지구대에서 가정폭력 관련 상담을 하던...
3,3,2020.10.22,광주지방법원 [2020노1868],재물손괴등[형사],피고인의 항소를 기각한다.,"... 동종 전과가 있고, 2017년 이후 여러 차례 폭행 범행을 저질렀으나 합의되..."
4,4,2020.10.22,광주지방법원 [2020고단3990],특수상해[형사],"피고인을 징역 8월에 처한다. 다만, 이 판결 확정일로부터 2년간 위 형의 집행을 ...",... 2년 피고인에게 폭력 범행으로... 피해자에게 가한 폭행의 방법과 정도가.....
...,...,...,...,...,...,...
4076,4076,2020.08.19,대전지방법원 천안지원 [2020고합53],강간미수[형사],... 집행을 유예한다. 피고인에게 40시간의 성폭력 치료강의 수강 및 200시간의...,".... 따라서 피고인이 폭행으로 피해자를... 경우, 피고인은 성폭력범죄의 처벌 ..."
4077,4077,2020.08.19,서울중앙지방법원 [2019고정1153],재물손괴등[형사],피고인을 벌금 130만 원에 처한다. 위 벌금을 납입하지 아니하는 경우 10만 원을...,"... 법정진술 1. F 작성의 진술서 1. 발생보고(폭행, 특수폭행), 112사건..."
4078,4078,2020.08.19,광주지방법원 순천지원 [2019고정459],폭행[형사],"피고인을 벌금 700,000원에 처한다. 피고인이 위 벌금을 납입하지 아니하는 경우...",... 피해자의 왼쪽 뺨을 1회 때리는 등 폭행하였다. 증거의 요지 1. 피고인의 ...
4079,4079,2020.08.19,춘천지방법원 속초지원 [2019고정79],재물손괴[형사],"피고인을 벌금 2,000,000원에 처한다. 피고인이 위 벌금을 납입하지 아니하는 ...","..., 동생인 피해자 C가 피고인이 모친에 대한 폭력 행사 여부를 확인하기 위하여..."
